In [23]:
import pandas as pd
import numpy as np
np.random.seed(0)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.svm import SVC  

from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

# EDA part 2:
- josh's confusion matrix predicted everything along Assault. all our falses were basically in Assault
    - Assault was 4200/5000, so this is to be expected w/ large class imbalance
    - 2 solutions:
        - more data
        - smote
- let's also look at doing SVC to pick out our variables
    

## SVM attempts

In [29]:
storage.head()

,agency,case_number,UCR_code,attempt_or_complete,DAC_1,location_type,weapon_type_1,bias_1,perp_seq_num,perp_age,...,num_ori_matches_x,state,victim_seq_num,ucr_1,vic_type,vic_age,vic_sex,vic_race,vic_ethn,num_ori_matches_y
0,AL0011200,2W1I0U7 6288,280,C,N,8,NaN,88,1,22.0,...,2.0,1,1,280,G,NaN,NaN,NaN,NaN,1.0
1,AL0011200,2W1I0U7 6288,280,C,N,8,NaN,88,2,30.0,...,NaN,1,1,280,G,NaN,NaN,NaN,NaN,1.0
2,AL0011200,2W1I0U7 62JW,13B,C,N,20,40,88,1,36.0,...,1.0,1,1,13B,I,39,M,W,N,1.0
3,AL0011200,2W1I0U7 64 8,220,C,N,20,NaN,88,0,NaN,...,1.0,1,1,220,I,30,F,W,N,1.0
4,AL0011200,2W1I0U7 64 W,35A,C,D,13,NaN,88,1,54.0,...,1.0,1,1,35A,S,NaN,NaN,NaN,NaN,1.0


In [2]:
print('importing')
df = pd.read_csv('merged.csv')
df = df.drop(['Unnamed: 0','state_y','state_x'], axis=1) # drop unneeded cols
# df = df.drop(to_drop,axis=1)

importing


/Users/mark/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8,15,21,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
storage = df # store a copy to save time re-importing

In [6]:
storage.dtypes

agency                  object
case_number             object
UCR_code                object
attempt_or_complete     object
DAC_1                   object
location_type            int64
weapon_type_1           object
bias_1                   int64
perp_seq_num             int64
perp_age               float64
perp_sex                object
perp_race               object
perp_ethn               object
num_ori_matches_x      float64
state                    int64
victim_seq_num           int64
ucr_1                   object
vic_type                object
vic_age                 object
vic_sex                 object
vic_race                object
vic_ethn                object
num_ori_matches_y      float64
dtype: object

In [7]:
storage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8505901 entries, 0 to 8505900
Data columns (total 23 columns):
agency                 object
case_number            object
UCR_code               object
attempt_or_complete    object
DAC_1                  object
location_type          int64
weapon_type_1          object
bias_1                 int64
perp_seq_num           int64
perp_age               float64
perp_sex               object
perp_race              object
perp_ethn              object
num_ori_matches_x      float64
state                  int64
victim_seq_num         int64
ucr_1                  object
vic_type               object
vic_age                object
vic_sex                object
vic_race               object
vic_ethn               object
num_ori_matches_y      float64
dtypes: float64(3), int64(5), object(15)
memory usage: 1.5+ GB


In [ ]:
# sm = SMOTE(random_state=1)
# X_train, y_train = sm.fit_resample(X_train, y_train)

In [16]:
df = storage

In [4]:
print('dropping nulls')
df = df.dropna()

print('begin sample')
# sample down to 30k
df = df.sample(5000, random_state=1)
print('sample complete')
# drop nulls?


dropping nulls
begin sample
sample complete


In [5]:
# df with dummies
print('getting dummies')
# remove Y to avoid Y dummies
target = df['UCR_code']
df = df.drop('UCR_code',axis=1)
print('dropped y')


getting dummies
dropped y


In [6]:
# dummify

df = pd.get_dummies(df,drop_first=True)
print('dummies acquired')
features = list(df.columns) # get dummy list

df['UCR_code'] = target # this should replace the original undummied Y
print('done with imports & cleaning')

dummies acquired
done with imports & cleaning


In [7]:

df.head(10)







,location_type,bias_1,perp_seq_num,perp_age,num_ori_matches_x,state,victim_seq_num,num_ori_matches_y,agency_AR0010100,agency_AR0010200,...,vic_sex_M,vic_sex_U,vic_race_B,vic_race_I,vic_race_P,vic_race_U,vic_race_W,vic_ethn_N,vic_ethn_U,UCR_code
3261531,14,88,1,34.0,2.0,24,1,1.0,0,0,...,1,0,0,0,0,0,1,0,1,13A
6262040,20,88,1,23.0,1.0,41,1,1.0,0,0,...,0,0,1,0,0,0,0,1,0,13B
6207246,20,88,1,59.0,1.0,41,1,1.0,0,0,...,0,0,1,0,0,0,0,1,0,13B
5617093,14,88,1,31.0,2.0,41,1,1.0,0,0,...,1,0,0,0,0,0,1,1,0,120
6260994,20,88,1,70.0,1.0,41,1,1.0,0,0,...,1,0,0,0,0,0,1,1,0,13B
6435132,20,88,1,22.0,1.0,42,1,1.0,0,0,...,1,0,0,0,0,0,1,0,0,13B
5566927,13,88,1,34.0,1.0,41,1,1.0,0,0,...,0,0,1,0,0,0,0,1,0,100
4588771,20,88,1,45.0,1.0,36,1,1.0,0,0,...,0,0,0,0,0,0,1,1,0,13B
6221231,13,88,1,66.0,1.0,41,1,1.0,0,0,...,0,0,1,0,0,0,0,1,0,13A
1502457,13,88,1,25.0,4.0,12,1,1.0,0,0,...,1,0,1,0,0,0,0,1,0,120


- 31,000 columns. Interesting stuff.
    - those aren't rookie numbers
    - we gotta get those numbers down

In [7]:
df['UCR_code'] = target

In [13]:
df_full['UCR_code'] = target

In [12]:
from time import time

In [22]:
# #
df_full = df

In [8]:
df = df.drop('UCR_code',axis=1)

In [9]:
df.columns

Index(['location_type', 'bias_1', 'perp_seq_num', 'perp_age',
       'num_ori_matches_x', 'state', 'victim_seq_num', 'num_ori_matches_y',
       'agency_AR0010100', 'agency_AR0010200',
       ...
       'vic_age_BB', 'vic_sex_M', 'vic_sex_U', 'vic_race_B', 'vic_race_I',
       'vic_race_P', 'vic_race_U', 'vic_race_W', 'vic_ethn_N', 'vic_ethn_U'],
      dtype='object', length=6152)

In [23]:
len(df)

10000

In [10]:
# make training splits
X = df
y = target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20) 

# let's get SMOTEing

In [25]:
!pip install imblearn

     |████████████████████████████████| 174kB 3.9MB/s eta 0:00:01


In [ ]:
sm = SMOTE(random_state=1) # build oversampling method
X_train, y_train = sm.fit_resample(X_train, y_train) # refit training data

In [22]:
df.columns[:10]

Index(['location_type', 'bias_1', 'perp_seq_num', 'perp_age',
       'num_ori_matches_x', 'state', 'victim_seq_num', 'num_ori_matches_y',
       'agency_AR0010100', 'agency_AR0010200'],
      dtype='object')

## SVC because we have *tons* of columns

In [13]:
from sklearn.svm import SVC  
tic = time()
svclassifier = SVC(kernel='rbf', C=0.05)  
svclassifier.fit(X_train, y_train) 
y_pred = svclassifier.predict(X_test)
toc = time()
print("run time is {} seconds".format(toc-tic))

/Users/mark/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


run time is 96.29190707206726 seconds


In [14]:
from sklearn.metrics import classification_report, confusion_matrix , accuracy_score
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred)) 
print("The accuracy score is" + " "+ str(accuracy_score(y_test, y_pred)))

[[  0   0   0   0   0   0   0   0   2   0   0   0]
 [  0   0   0   0   0   0   0   0  15   0   0   0]
 [  0   0   0   0   0   0   0   0  34   0   0   0]
 [  0   0   0   0   0   0   0   0   4   0   0   0]
 [  0   0   0   0   0   0   0   0   1   0   0   0]
 [  0   0   0   0   0   0   0   0  36   0   0   0]
 [  0   0   0   0   0   0   0   0  70   0   0   0]
 [  0   0   0   0   0   0   0   0 211   0   0   0]
 [  0   0   0   0   0   0   0   0 605   0   0   0]
 [  0   0   0   0   0   0   0   0   1   0   0   0]
 [  0   0   0   0   0   0   0   0  20   0   0   0]
 [  0   0   0   0   0   0   0   0   1   0   0   0]]
              precision    recall  f1-score   support

         09A       0.00      0.00      0.00         2
         100       0.00      0.00      0.00        15
         11A       0.00      0.00      0.00        34
         11B       0.00      0.00      0.00         4
         11C       0.00      0.00      0.00         1
         11D       0.00      0.00      0.00        36
        

/Users/mark/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [15]:
len(storage)

8505901

# SVC again, n=10,000

above cells in one cell for running. adjust sample size according
- really need to put this in a function to save time

In [16]:
df = storage

print('dropping nulls')
df = df.dropna()

print('begin sample')
# sample down to 30k
df = df.sample(10000, random_state=1)
print('sample complete')
# drop nulls?
# df with dummies
print('getting dummies')
# remove Y to avoid Y dummies
target = df['UCR_code']
df = df.drop('UCR_code',axis=1)
print('dropped y')

# dummify

df = pd.get_dummies(df,drop_first=True)
print('dummies acquired')
features = list(df.columns) # get dummy list

df['UCR_code'] = target # this should replace the original undummied Y
print('done with imports & cleaning')
df_full = df # checkpoint
df = df.drop('UCR_code',axis=1)

# make training splits
X = df
y = target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20) 

dropping nulls
begin sample
sample complete
getting dummies
dropped y
dummies acquired
done with imports & cleaning


In [17]:
tic = time()
svclassifier = SVC(kernel='rbf', C=0.05)  
svclassifier.fit(X_train, y_train) 
y_pred = svclassifier.predict(X_test)
toc = time()
print("run time is {} seconds".format(toc-tic))

/Users/mark/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


run time is 696.6429498195648 seconds


In [18]:
from sklearn.metrics import classification_report, confusion_matrix , accuracy_score
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred)) 
print("The accuracy score is" + " "+ str(accuracy_score(y_test, y_pred)))

[[   0    0    0    0    0    0    0    0    8    0    0    0]
 [   0    0    0    0    0    0    0    0   25    0    0    0]
 [   0    0    0    0    0    0    0    0   54    0    0    0]
 [   0    0    0    0    0    0    0    0   15    0    0    0]
 [   0    0    0    0    0    0    0    0    7    0    0    0]
 [   0    0    0    0    0    0    0    0   55    0    0    0]
 [   0    0    0    0    0    0    0    0  120    0    0    0]
 [   0    0    0    0    0    0    0    0  388    0    0    0]
 [   0    0    0    0    0    0    0    0 1294    0    0    0]
 [   0    0    0    0    0    0    0    0    1    0    0    0]
 [   0    0    0    0    0    0    0    0   32    0    0    0]
 [   0    0    0    0    0    0    0    0    1    0    0    0]]
              precision    recall  f1-score   support

         09A       0.00      0.00      0.00         8
         100       0.00      0.00      0.00        25
         11A       0.00      0.00      0.00        54
         11B       0.00   

/Users/mark/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Accuracy boosted by ~.05 when n doubled to 10,000
However, classification matrix is still only guessing Assault due to class imbalance

## let's try smote

In [26]:
len(df)

10000

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)  # from prior 10k model

In [22]:
sm = SMOTE(random_state=1)
X_train, y_train = sm.fit_resample(X_train, y_train)

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 2, n_neighbors = 6

In [24]:
tic = time()
svclassifier = SVC(kernel='rbf', C=0.05)  
svclassifier.fit(X_train, y_train) 
y_pred = svclassifier.predict(X_test)
toc = time()
print("run time is {} seconds".format(toc-tic))

/Users/mark/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


run time is 728.5693261623383 seconds


In [25]:
from sklearn.metrics import classification_report, confusion_matrix , accuracy_score
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred)) 
print("The accuracy score is" + " "+ str(accuracy_score(y_test, y_pred)))

[[   0    0    0    0    0    0    0    0    0    6    0    0    0]
 [   0    0    0    0    0    0    0    0    0    2    0    0    0]
 [   0    0    0    0    0    0    0    0    0   27    0    0    0]
 [   0    0    0    0    0    0    0    0    0   71    0    0    0]
 [   0    0    0    0    0    0    0    0    0   13    0    0    0]
 [   0    0    0    0    0    0    0    0    0    4    0    0    0]
 [   0    0    0    0    0    0    0    0    0   51    0    0    0]
 [   0    0    0    0    0    0    0    0    0  116    0    0    0]
 [   0    0    0    0    0    0    0    0    0  397    0    0    0]
 [   0    0    0    0    0    0    0    0    0 1272    0    0    0]
 [   0    0    0    0    0    0    0    0    0    2    0    0    0]
 [   0    0    0    0    0    0    0    0    0   38    0    0    0]
 [   0    0    0    0    0    0    0    0    0    1    0    0    0]]
              precision    recall  f1-score   support

         09A       0.00      0.00      0.00         6
      

/Users/mark/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


3261531    13A
6262040    13B
6207246    13B
5617093    120
6260994    13B
          ... 
601657     13B
793863     11D
3289739    13B
7809209    13B
5699603    13B
Name: UCR_code, Length: 10000, dtype: object

In [26]:
len(df)

30000

- note on extra info & ethical considerations:
    - i'm gonna need to have a look at my false positives & false negatives
        - i included criminal & victim demographics in this model, so it's open to the same sort of bias that people criticized that hearing/sentencing software over
        - after running models, examine distribution of errors (both types) among racial groups
            - is my model racist?
                - if the data is racist, then it probably will be

In [30]:
import xgboost as xgb

In [31]:
import pandas as pd
import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [32]:
df.dtypes[:10] # looks like we've got only numeric types, so we can go ahead and XGB

location_type          int64
bias_1                 int64
perp_seq_num           int64
perp_age             float64
num_ori_matches_x    float64
state                  int64
victim_seq_num         int64
num_ori_matches_y    float64
agency_AR0010100       uint8
agency_AR0010200       uint8
dtype: object

In [33]:
target

3261531    13A
6262040    13B
6207246    13B
5617093    120
6260994    13B
          ... 
601657     13B
793863     11D
3289739    13B
7809209    13B
5699603    13B
Name: UCR_code, Length: 10000, dtype: object

In [34]:


X_train, X_test, y_train, y_test = train_test_split(df, target)

In [35]:
df.dtypes.value_counts()

uint8      11347
int64          5
float64        3
dtype: int64

In [36]:
df.dtypes[:15]

location_type          int64
bias_1                 int64
perp_seq_num           int64
perp_age             float64
num_ori_matches_x    float64
state                  int64
victim_seq_num         int64
num_ori_matches_y    float64
agency_AR0010100       uint8
agency_AR0010200       uint8
agency_AR0020100       uint8
agency_AR0020200       uint8
agency_AR0030000       uint8
agency_AR0030100       uint8
agency_AR0040000       uint8
dtype: object

In [37]:
storage = df

In [38]:
df = df.drop('UCR_code',axis=1)

KeyError: "['UCR_code'] not found in axis"

In [39]:
storage_2 = df
df = storage_2.sample(5000, random_state=1)

In [ ]:
clf = xgb.XGBClassifier()
clf.fit(X_train, y_train)
training_preds = clf.predict(X_train)
val_preds = clf.predict(X_test)
training_accuracy = accuracy_score(y_train, training_preds)
val_accuracy = accuracy_score(y_test, val_preds)

print("Training Accuracy: {:.4}%".format(training_accuracy * 100))
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))

In [15]:
'UCR_code' in list(df.columns)

True